# Modul 6 Persamaan Diferensial Numerik: Masalah Nilai Batas untuk PDB

Kembali ke [Persamaan Diferensial Numerik](./pdnum2024genap.qmd)

Di modul ini, kita akan membahas metode-metode untuk masalah nilai batas untuk PDB, yaitu:

1. *Linear Shooting*
2. *Nonlinear Shotting*
3. *Finite Difference* linier
4. *Finite Difference* nonlinier

## Review: Runge-Kutta orde 4 untuk sistem

*Shooting method* untuk masalah nilai batas melibatkan sistem persamaan diferensial. Kita akan menggunakan kode metode Runge-Kutta orde 4 untuk sistem dari modul sebelumnya:

In [ ]:
function [t, w] = rko4_sysm(cell_f, a, b, N, alphas)
  m = length(cell_f);

  h = (b - a) / N;
  t = zeros(N + 1, 1);
  w = zeros(m, N + 1);
  t(1) = a;
  w(:, 1) = alphas;

  k1 = zeros(m, 1);
  k2 = zeros(m, 1);
  k3 = zeros(m, 1);
  k4 = zeros(m, 1);
  for i = 1 : N
    t(i + 1) = t(i) + h;

    for j = 1 : m
      k1(j) = h * cell_f{j}(t(i), w(:, i));
    endfor

    for j = 1 : m
      k2(j) = h * cell_f{j}(t(i) + (h / 2), w(:, i) + (k1 / 2));
    endfor

    for j = 1 : m
      k3(j) = h * cell_f{j}(t(i) + (h / 2), w(:, i) + (k2 / 2));
    endfor

    for j = 1 : m
      k4(j) = h * cell_f{j}(t(i + 1), w(:, i) + k3);
    endfor

    for j = 1 : m
      w(j, i + 1) = w(j, i) + (k1(j) + 2 * k2(j) + 2 * k3(j) + k4(j)) / 6;
    endfor
  endfor
endfunction

Sebenarnya tidak harus metode Runge-Kutta orde 4 untuk sistem. Boleh ditukar dengan metode lainnya untuk sistem, misalnya metode Adams predictor-corrector orde 4 untuk sistem.

## Linear Shooting Method

### Bentuk umum, ide utama, penyederhanaan

Linear Shooting merupakan metode untuk menyelesaikan sejenis masalah nilai batas untuk PDB, yaitu yang berbentuk:

$y'' = f\left(x,y,y'\right) = p(x)y' + q(x)y + r(x), \;a\leq x\leq b$

$y(a)=\alpha, \;y(b)=\beta$

dengan

- $p(x), q(x), r(x)$ adalah fungsi kontinu dalam $x$
- $q(x) > 0$ pada $[a,b]$ agar dijamin ada solusi unik

Cara penyelesaiannya:

1. Selesaikan MNA PDB orde 2 berikut, solusinya disebut $y_1 \left(x\right)$:

$$y'' = p(x)y' + q(x)y + r(x), \quad a \le x \le b, \quad y\left(a\right) = \alpha, \quad y'\left(a\right) = 0$$

2. Selesaikan MNA PDB orde 2 berikut, solusinya disebut $y_2 \left(x\right)$

$$y'' = p(x)y' + q(x)y, \quad a \le x \le b, \quad y\left(a\right) = 0, \quad y'\left(a\right) = 1$$

3. Solusi akhirnya adalah

$$y\left(x\right) = y_1 \left(x\right) + \frac{\beta - y_1 \left(b\right)}{y_2 \left(b\right)} y_2 \left(x\right)$$

Kita bisa menuliskan kedua MNA PDB orde 2 tersebut masing-masing sebagai sistem PDB orde 1, seperti biasa dengan permisalan $u_1(x) = y(x)$ dan $u_2(x) = y'(x)$.

Sehingga, langkahnya menjadi:

1. Selesaikan sistem PDB orde 1 berikut. Kemudian solusi $u_1(x)$ disebut $y_1(x)$ dan solusi $u_2(x)$ disebut $y_1'(x)$.

\begin{aligned}
u_1'(x) &= u_2(x) \\
u_2'(x) &= p(x) u_2(x) + q(x) u_1(x) + r(x) \\
u_1 (a) &= \alpha, \quad u_2 (a) = 0
\end{aligned}

2. Selesaikan sistem PDB orde 1 berikut. Kemudian solusi $u_1(x)$ disebut $y_2(x)$ dan solusi $u_2(x)$ disebut $y_2'(x)$.

\begin{aligned}
u_1'(x) &= u_2(x) \\
u_2'(x) &= p(x)u_2(x) + q(x)u_1(x) \\
u_1 (a) &= 0, \quad u_2(a) = 1
\end{aligned}

3. Solusi akhirnya adalah

$$y(x) = y_1(x) + \frac{\beta - y_1(b)}{y_2(b)} y_2(x)$$

Kalau perlu,

$$y'(x) = y_1'(x) + \frac{\beta - y_1(b)}{y_2(b)} y_2'(x)$$

### Function file (dari pseudocode di buku)

In [ ]:
function [x_i, w_1i, w_2i] = linshoot_pseudocode(p, q, r, a, b, n, alpha, beta)
  h = (b - a)/n;
  u = [alpha ; 0];
  v = [0 ; 1];
  x_i = w_1i = w_2i = [];
  for i = 1:n
    x = a + (i-1)*h;

    k_11 = h * u(2,i);
    k_12 = h * (p(x)*u(2,i) + q(x)*u(1,i) + r(x));

    k_21 = h * (u(2,i)+(k_12/2));
    k_22 = h * (p(x+(h/2))*(u(2,i)+(k_12/2)) + q(x+(h/2))*(u(1,i)+(k_11/2)) + r(x+(h/2)));

    k_31 = h * (u(2,i)+(k_22/2));
    k_32 = h * (p(x+(h/2))*(u(2,i)+(k_22/2)) + q(x+(h/2))*(u(1,i)+(k_21/2)) + r(x+(h/2)));

    k_41 = h * (u(2,i)+k_32);
    k_42 = h * (p(x+h)*(u(2,i)+k_32) + q(x+h)*(u(1,i)+k_31) + r(x+h));

    u(1,i+1) = u(1,i) + ((k_11 + 2*k_21 + 2*k_31 + k_41)/6);
    u(2,i+1) = u(2,i) + ((k_12 + 2*k_22 + 2*k_32 + k_42)/6);

    kp_11 = h * v(2,i);
    kp_12 = h * (p(x)*v(2,i) + q(x)*v(1,i));

    kp_21 = h * (v(2,i) + (kp_12/2));
    kp_22 = h * (p(x+(h/2))*(v(2,i)+(kp_12/2)) + q(x+(h/2))*(v(1,i)+(kp_11/2)));

    kp_31 = h * (v(2,i)+(kp_22/2));
    kp_32 = h * (p(x+(h/2))*(v(2,i)+(kp_22/2)) + q(x+(h/2))*(v(1,i)+(kp_21/2)));

    kp_41 = h * (v(2,i)+kp_32);
    kp_42 = h * (p(x+h)*(v(2,i)+kp_32) + q(x+h)*(v(1,i)+kp_31));

    v(1,i+1) = v(1,i) + (kp_11 + 2*kp_21 + 2*kp_31 + kp_41)/6;
    v(2,i+1) = v(2,i) + (kp_12 + 2*kp_22 + 2*kp_32 + kp_42)/6;
  endfor

  w = [alpha ; ((beta - u(1,(n+1))) / v(1,(n+1)))];
  x_i(1) = a;
  w_1i(1) = w(1,1);
  w_2i(1) = w(2,1);

  for i = 2:(n+1)
    W1 = u(1,i) + w(2,1)*v(1,i);
    W2 = u(2,i) + w(2,1)*v(2,i);
    x = a + (i-1)*h;
    x_i(i) = x;
    w_1i(i) = W1;
    w_2i(i) = W2;
  endfor
endfunction

### Function file (lebih sederhana)

In [ ]:
function [x, w1, w2] = linear_shooting(p, q, r, a, b, N, alph, bet)
  % sistem PDB yang pertama
  u1_aksen = @(x, u) u(2);
  u2_aksen = @(x, u) p(x)*u(2) + q(x)*u(1) + r(x);
  [x, w_pers1] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [alph, 0]);
  y1_b = w_pers1(1, N+1);
  
  % sistem PDB yang kedua
  u1_aksen = @(x, u) u(2);
  u2_aksen = @(x, u) p(x)*u(2) + q(x)*u(1);
  [x, w_pers2] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [0, 1]);
  y2_b = w_pers2(1, N+1);
  
  % solusi akhir (superposisi)
  w_akhir = w_pers1 + (bet - y1_b)/(y2_b) * w_pers2;
  % dipisah jadi w1i (aproksimasi y(x)) dan w2i (aproksimasi y'(x))
  [w1, w2] = w_akhir';
endfunction

### Contoh Linear Shooting

$y'' = -\frac{2}{x}y' + \frac{2}{x^2}y + \frac{\sin(\ln(x))}{x^2}, \; 1\leq x\leq 2$

$y(1)=1,\; y(2)=2$

dengan $N=10$

dan solusi eksak:

$y=c_1x+\frac{c_2}{x^2} - \frac{3}{10}\sin(\ln(x))-\frac{1}{10}cos(\ln(x))$

$c_2 = \frac{1}{70}(8-12\sin(\ln(2)) - 4\cos(\ln(2)))$

$c_1 = \frac{11}{10}-c_2$

Berikut code script file untuk permasalahan di atas menggunakan metode linear shooting:

In [ ]:
p = @(x) (-2*(x^(-1)));
q = @(x) (2*(x^(-2)));
r = @(x) (sin(log(x))*(x^(-2)));
a = 1;
b = 2;
N = 10;
alph = 1;
bet = 2;

[xi, w1i, w2i] = linear_shooting(p, q, r, a, b, N, alph, bet);

c2 = (8-12*sin(log(2)) - 4*cos(log(2)))/70;
c1 = (11/10) - c2;
sln = @(x) (c1*x + (c2*x^(-2)) - (3/10)*sin(log(x)) - (1/10)*cos(log(x)));
w = [];
for i = 1:length(xi)
  w(i) = sln(xi(i));
endfor

[xi', w1i', w']

hold on;
fplot(sln, [1,2], 'k');
scatter(xi, w1i, '-r');
legend('Eksak', 'Aproksimasi');
legend('location', 'northwest');

Jika kita run script file tersebut, maka program akan mengeluarkan dua macam output, yaitu tabel serta plot perbandingan solusi eksak dan aproksimasi seperti di bawah ini:

![image.png](attachment:image.png)

![image-2.png](attachment:image-2.png)

## Nonlinear Shooting Method

### Bentuk umum, ide utama

Nonlinear Shooting digunakan untuk menyelesaikan masalah PD berbentuk:

$y'' = f(x, y, y'), \; a\leq x \leq b$

$y(a)=\alpha, \; y(b)=\beta$

dengan $f$ boleh berupa fungsi linier maupun nonlinier

Cara penyelesaiannya:

1. Tentukan toleransi $\varepsilon$, dan pilih tebakan awal $t_0$ (yaitu $t_k$ sebelum iterasi pertama, yaitu dengan $k=0$). Kalau bingung, disarankan

$$t_0 = \frac{\beta - \alpha}{b-a}$$

2. Selesaikan MNA PDB orde 2 berikut, misalkan solusinya disebut $w(x,t_k)$:

$$y'' = f(x,y,y'), \quad a \le x \le b, \quad y(a) = \alpha, \quad y'(a) = t_k$$

3. Periksa apakah $\left|w(x,t_k) - \beta\right| \le \varepsilon$.

    - Kalau iya, selesai; solusi akhirnya adalah $y(x) = w(x,t_k)$.
    - Kalau tidak, peroleh tebakan baru untuk $t_i$ (misalnya dengan metode secant atau metode Newton), lalu kembali ke langkah 2.

Seperti biasa, kita bisa misalkan $u_1(x) = y(x)$ dan $u_2(x) = y'(x)$ agar MNA PDB orde 2 menjadi sistem PDB orde 1.

Cara penyelesaiannya menjadi:

1. Tentukan toleransi $\varepsilon$, dan pilih tebakan awal $t_0$ (yaitu $t_k$ sebelum iterasi pertama, yaitu dengan $k=0$). Kalau bingung, disarankan

$$t_0 = \frac{\beta - \alpha}{b-a}$$

2. Selesaikan sistem PDB orde 1 berikut. Kemudian $u_1(x)$ disebut $w(x,t_k)$ dan $u_2(x)$ disebut $w'(x,t_k)$.

\begin{aligned}
u_1'(x) &= u_2(x) \\
u_2'(x) &= f(x,u_1,u_2) \\
y(a) &= \alpha, \quad y'(a) = t_k
\end{aligned}

3. Periksa apakah $\left|w(x,t_k) - \beta\right| \le \varepsilon$.

    - Kalau iya, selesai; solusi akhirnya adalah $y(x) = w(x,t_k)$.
    - Kalau tidak, peroleh tebakan baru untuk $t_k$ (misalnya dengan metode secant atau metode Newton), lalu kembali ke langkah 2.

### Function file (metode secant)

In [ ]:
function [x, w1, w2] = nonlinear_shooting_secant(f, a, b, N, alph, bet, tol, t0, t1)
  u1_aksen = @(x, u) u(2);
  u2_aksen = @(x, u) f(x, u(1), u(2));
  
  t_k_min_2 = t0;
  t_k_min_1 = t1;
  [x, w_k_min_2] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [alph, t_k_min_2]);
  [x, w_k_min_1] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [alph, t_k_min_1]);
  w_k = w_k_min_1;
  err = abs(w_k(1, N+1) - bet);
  while !(err <= tol)
    pembilang = (w_k_min_1(1,N+1) - bet) * (t_k_min_1 - t_k_min_2);
    penyebut = w_k_min_1(1,N+1) - w_k_min_2(1,N+1);
    t_k = t_k_min_1 - pembilang/penyebut;

    t_k_min_2 = t_k_min_1
    t_k_min_1 = t_k

    [x, w_k] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [alph, t_k]);
    err = abs(w_k(1, N+1) - bet);
  endwhile
  % keluar loop artinya toleransi sudah terpenuhi
  
  % memisahkan w_k menjadi w1i dan w2i
  [w1, w2] = w_k';
endfunction

### Modifikasi untuk metode Newton

Untuk menggunakan metode Newton, diperlukan tidak hanya $y(b,t)$ tetapi juga turunannya $\frac{\partial y(b,t)}{\partial t}$ yang sayangnya tidak dimiliki.

Setelah penjabaran yang panjang di buku, ternyata bisa dimisalkan

$$z(x,t) = \frac{\partial y(x,t)}{\partial t}$$

dan nilai fungsi $z$ ini ternyata bisa diperoleh dengan menyelesaikan suatu MNA PDB orde 2 (lagi). Sehingga, di tiap iterasi, ada dua MNA PDB orde 2 yang harus diselesaikan.

Langkah *nonlinear shooting* dengan metode Newton bisa ditulis:

1. Hitung rumus $\frac{\partial f}{\partial y}(x,y,y')$ dan rumus $\frac{\partial f}{\partial y'}(x,y,y')$ secara analitik.

2. Tentukan toleransi $\varepsilon$, dan pilih tebakan awal $t_0$ (yaitu $t_k$ sebelum iterasi pertama, yaitu dengan $k=0$). Kalau bingung, disarankan

$$t_0 = \frac{\beta - \alpha}{b-a}$$

3. Selesaikan MNA PDB orde 2 berikut, misalkan solusinya disebut $w(x,t_k)$:

$$y'' = f(x,y,y'), \quad a \le x \le b, \quad y(a) = \alpha, \quad y'(a) = t_k$$

4. Selesaikan MNA PDB orde 2 berikut:

$$z'' = \frac{\partial f}{\partial y}(x,y,y')z(x) + \frac{\partial f}{\partial y'}(x,y,y')z'(x), \quad z(a) = 0, \quad z'(a) = 1$$

5. Periksa apakah $\left|w(x,t_k) - \beta\right| \le \varepsilon$.

    - Kalau iya, selesai; solusi akhirnya adalah $y(x) = w(x,t_k)$.
    - Kalau tidak, kembali ke langkah 3 setelah memperoleh tebakan baru untuk $t_k$:
    $$t_k = t_{k-1} - \frac{w(b, t_{k-1}) - \beta}{z(b, t_{k-1})}$$

Dengan permisalan $u_1$ dan $u_2$ agar PDB orde 2 menjadi sistem PDB orde 1, langkah-langkahnya menjadi:

1. Hitung rumus $\frac{\partial f}{\partial y}(x,y,y')$ dan rumus $\frac{\partial f}{\partial y'}(x,y,y')$ secara analitik.

2. Tentukan toleransi $\varepsilon$, dan pilih tebakan awal $t_0$ (yaitu $t_k$ sebelum iterasi pertama, yaitu dengan $k=0$). Kalau bingung, disarankan

$$t_0 = \frac{\beta - \alpha}{b-a}$$

3. Selesaikan sistem PDB orde 1 berikut. Kemudian $u_1(x)$ disebut $w(x,t_k)$ dan $u_2(x)$ disebut $w'(x,t_k)$.

\begin{aligned}
u_1'(x) &= u_2(x) \\
u_2'(x) &= f(x,u_1,u_2) \\
y(a) &= \alpha, \quad y'(a) = t_k
\end{aligned}

4. Selesaikan sistem PDB orde 1 berikut. Kemudian $u_1(x)$ disebut $z(x,t_k)$ dan $u_2(x)$ disebut $z'(x,t_k)$.

\begin{aligned}
u_1'(x) &= u_2(x) \\
u_2'(x) &= \frac{\partial f}{\partial y}(x,u_1,u_2)u_1(x) + \frac{\partial f}{\partial y'}(x,u_1,u_2)u_2(x) \\
u_1(a) &= 0, \quad u_2(a) = 1
\end{aligned}

5. Periksa apakah $\left|w(x,t_k) - \beta\right| \le \varepsilon$.

    - Kalau iya, selesai; solusi akhirnya adalah $y(x) = w(x,t_k)$.
    - Kalau tidak, kembali ke langkah 3 setelah memperoleh tebakan baru untuk $t_k$:
    $$t_k = t_{k-1} - \frac{w(b, t_{k-1}) - \beta}{z(b, t_{k-1})}$$

### Function file (dari pseudocode)

In [ ]:
function [x_i, w_1i, w_2i] = nonlinshoot_pseudocode(f, fy, fyp, a, b, n, alpha, beta, m, tol)
  % m adalah maksimum iterasi

  h = (b - a)/n;
  k = 1;
  tk = (beta - alpha)/(b - a);
  x_i = w_1i = w_2i = [];
  while k <= m
    w = [alpha;tk];
    u = [0,1];
    for i = 1:n
      x = a + (i-1)*h;

      k_11 = h*w(2,i);
      k_12 = h*f(x, w(1,i), w(2,i));

      k_21 = h*(w(2,i)+(k_12/2));
      k_22 = h*f((x+(h/2)), (w(1,i)+(k_11/2)), (w(2,i)+(k_12/2)));

      k_31 = h*(w(2,i)+(k_22/2));
      k_32 = h*f((x+(h/2)), (w(1,i)+(k_21/2)), (w(2,i)+(k_22/2)));

      k_41 = h*(w(2,i)+k_32);
      k_42 = h*f((x+h), (w(1,i)+k_31), (w(2,i)+k_32));

      w(1,i+1) = w(1,i) + ((k_11 + 2*k_21 + 2*k_31 + k_41)/6);
      w(2,i+1) = w(2,i) + ((k_12 + 2*k_22 + 2*k_32 + k_42)/6);

      kp_11 = h*u(2);
      kp_12 = h*(fy(x, w(1,i), w(2,i))*u(1) + fyp(x, w(1,i), w(2,i))*u(2));

      kp_21 = h*(u(2) + (kp_12/2));
      kp_22 = h*(fy((x+(h/2)), w(1,i), w(2,i))*u(1) + fyp((x+(h/2)), w(1,i), w(2,i))*(u(2) + (kp_12/2)));

      kp_31 = h*(u(2)+(kp_22/2));
      kp_32 = h*(fy((x+(h/2)), w(1,i), w(2,i))*(u(1) + (kp_21/2)) + fyp((x+(h/2)), w(1,i), w(2,i))*(u(2) + (kp_22/2)));

      kp_41 = h*(u(2)+kp_32);
      kp_42 = h*(fy((x+h), w(1,i), w(2,i))*(u(1)+kp_31) + fyp((x+h), w(1,i), w(2,i))*(u(2) + kp_32));

      u(1) = u(1) + (kp_11 + 2*kp_21 + 2*kp_31 + kp_41)/6;
      u(2) = u(2) + (kp_12 + 2*kp_22 + 2*kp_32 + kp_42)/6;
    endfor

  if abs(w(1,n+1) - beta) <= tol       % jika sudah mencapai batas toleransi maka program berhenti
    for i = 1:(n+1)
      x = a+(i-1)*h;
      x_i(i) = x;
      w_1i(i) = w(1,i);
      w_2i(i) = w(2,i);
    endfor
    return
  endif
  tk = tk-((w(1,n+1) - beta)/u(1));
  k = k + 1;
  endwhile
  disp('max iteration')
endfunction

### Function file (metode Newton)

In [ ]:
function [x, w1, w2] = nonlinear_shooting_newton(f, fy, fyp, a, b, N, alph, bet, tol, t0)
  % kalau input t0 bukan angka, dianggap tidak memilih tebakan awal
  if isnumeric(t0)
    t_k = t0;
  else
    t_k = (bet-alph)/(b-a);
  endif

  err = tol + 1;
  while !(err <= tol)
    % selesaikan sistem pertama
    u1_aksen = @(x, u) u(2);
    u2_aksen = @(x, u) f(x, u(1), u(2));
    [x, w_sys] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [alph, t_k]);

    % selesaikan sistem kedua
    u1_aksen = @(x, u) u(2);
    u2_aksen = @(x, u) fy(x, u(1), u(2))*u(1) + fyp(x, u(1), u(2))*u(2);
    [x, z_sys] = rko4_sysm({u1_aksen, u2_aksen}, a, b, N, [0, 1]);

    % periksa toleransi, update t_k
    err = abs(w_sys(1, N+1) - bet);
    if !(err <= tol)
      t_k = t_k - (w_sys(1, N+1) - bet)/(z_sys(1, N+1));
    endif
  endwhile
  % keluar loop artinya toleransi sudah terpenuhi

  % pisahkan w_sys menjadi w1i dan w2i
  [w1, w2] = w_sys';
endfunction

### Contoh Nonlinear Shooting

$y'' = \frac{1}{8}(32+2x^3-yy'), \; 1\leq x \leq 3$

$y(1) = 17, \; y(3)=43/3$

dengan $N=20$, $m=10$, dan toleransi $=10^{-5}$

dan solusi eksak:

$y(x)=x^2 + \frac{16}{x}$

Hint:

\begin{aligned}
f(x,y,y') &= \frac{1}{8}(32+2x^3-yy') \\
\frac{\partial f}{\partial y}(x,y,y') &= -\frac{1}{8}y' \\
\frac{\partial f}{\partial y'}(x,y,y') &= -\frac{1}{8}y
\end{aligned}

Berikut code script file untuk permasalahan di atas menggunakan metode linear shooting:

In [ ]:
f = @(x, y, yp) ((1/8)*(32 + 2*x^3 - y*yp));
fy = @(x, y, yp) (-yp/8);
fyp = @(x, y, yp) (-y/8);
a = 1;
b = 3;
N = 20;
alph = 17;
bet = 43/3;
tol = 10^(-5);

[xi, w1i, w2i] = nonlinear_shooting_newton(f, fy, fyp, a, b, N, alph, bet, tol, "");

sln = @(x) ((x^2) + (16/x));
w = [];
for i = 1:length(xi)
  w(i) = sln(xi(i));
endfor

[xi', w1i', w']

hold on;
fplot(sln, [1,3], 'k');
scatter(xi, w1i, 'r');
legend('Eksak', 'Aproksimasi');

Jika kita run script file tersebut, maka program akan mengeluarkan dua macam output, yaitu tabel serta plot perbandingan solusi eksak dan aproksimasi seperti di bawah ini:

![image.png](attachment:image.png)

![image-2.png](attachment:image-2.png)